# Build a variational autoencoder

### References

https://blog.keras.io/building-autoencoders-in-keras.html

https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
from keras import backend
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegBowVectorizer

### Data generation

In [2]:
n_train = 10000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegBowVectorizer()

train_strs = [g.get_reg()[1:] for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
test_strs = [g.get_reg()[1:] for _ in range(n_test)]
test_strs = [x for x in test_strs if x not in set(train_strs)]  # sanity check
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print('input vector length = ' + str(vec_length))
print('actual number of test samples = ' + str(len(test_strs)))

input vector length = 216
actual number of test samples = 1000


### Variational autoencoder

In [3]:
latent_dim = 100
intermediate_dim = 400
original_dim = vec_length

### Define the encoder
inputs = keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
h = layers.Dense(intermediate_dim, activation='relu')(h)
h = layers.Dense(intermediate_dim, activation='relu')(h)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = backend.random_normal(shape=(backend.shape(z_mean)[0], latent_dim),
                                    mean=0., stddev=0.1)
    return z_mean + backend.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

### Define the decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
# x = layers.Dense(intermediate_dim, activation='relu')(x)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

### Define the loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - backend.square(z_mean) - backend.exp(z_log_sigma)
kl_loss = backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

In [ ]:
vae.fit(train_vecs, train_vecs,
        epochs=50,
        batch_size=256,
        shuffle=True,
        validation_data=(test_vecs, test_vecs))

Epoch 1/50
40/40 [==============================] - 2s 59ms/step - loss: 119.5673 - val_loss: 96.4804
Epoch 2/50
40/40 [==============================] - 2s 51ms/step - loss: 85.9454 - val_loss: 77.7067
Epoch 3/50
40/40 [==============================] - 2s 56ms/step - loss: 72.3342 - val_loss: 67.8532
Epoch 4/50
40/40 [==============================] - 2s 59ms/step - loss: 64.7021 - val_loss: 62.1524
Epoch 5/50
40/40 [==============================] - 2s 53ms/step - loss: 59.9481 - val_loss: 58.1226
Epoch 6/50
40/40 [==============================] - 2s 53ms/step - loss: 56.4716 - val_loss: 55.1747
Epoch 7/50
40/40 [==============================] - 3s 63ms/step - loss: 53.7989 - val_loss: 52.7708
Epoch 8/50
40/40 [==============================] - 3s 64ms/step - loss: 51.1698 - val_loss: 49.8286
Epoch 9/50
40/40 [==============================] - 2s 59ms/step - loss: 48.6635 - val_loss: 47.6680
Epoch 10/50
40/40 [==============================] - 3s 73ms/step - loss: 46.7520 - val_lo

In [ ]:
decoded_regs = vae.predict(test_vecs)
print(test_strs[:10])
print([v.recover(x) for x in decoded_regs[:10]])
acc = np.sum([v.recover(x) == y for x, y in zip(decoded_regs, test_strs)]) / len(test_strs)
print('accuracy = ' + str(acc))